# ID2214/FID3214 Assignment 2 Group no. ID2214 - 5
### Project members:

Lukas Olenborg,
Charlotte Jacquet,
Isabella Rositi

## Load NumPy, pandas and time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Filepath to shared drive (you have to add the 'Programming for Data Science' folder to your drive)
filepath = '/content/drive/MyDrive/Programming for Data Science/Assignment 2/'

# Filepath to your own drive folder and files if above doesnt work
#filepath = '/content/drive/MyDrive/Colab Notebooks/ID2214/'

In [ ]:
# Filepath for TA, uncomment when submitting
#filepath = ''

In [ ]:
import numpy as np
import pandas as pd
import time
import math as mat

In [ ]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.8.15
NumPy version: 1.21.6
Pandas version: 1.3.5


## Reused functions from Assignment 1

In [ ]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# FILTERING
def create_column_filter(df):

    # copy the input dataframe into df_copy
    df_copy = df.copy()

    for colname, values in df.iteritems():
        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue
        else:
            # Drop a column if it contains only missing values
            df_copy = df_copy.dropna(how = 'all', axis=1)

            # Drop a column if there's only one unique value...
            arr = df[colname].unique()
            # ... apart from the missing values
            arr = arr[~ pd.isnull(arr)] # '~' = np not operator (!), isnull = True where nan
            if arr.size == 1:
                #if colname in df_copy.columns: # this line replaced with errors='ignore' below
                df_copy = df_copy.drop([colname], axis=1, errors='ignore')
            #alternative apart from the missing values
            # elif arr.size == 2:
            #   for i in range(arr.size):
            #       if np.isnan(arr[i]):
            #           df_copy = df_copy.drop([colname],axis=1, errors='ignore')


    column_filter = df_copy.columns.values.tolist()

    return df_copy, column_filter

def apply_column_filter(df, column_filter):
    return df.loc[:,column_filter] # : for all rows , column_filter cols


#IMPUTATION
def create_imputation(df):
    dfcpy = df.copy()
    imputation = {}

    for col in df.columns:
        if col == "ID" or col == "CLASS":
            continue

        coltype = dfcpy[col].dtypes # get type of column

        if coltype == "int64" or coltype == "float64":
            # Replace nan values with mean of column for numeric cols
            mean = np.mean(dfcpy[col])

            # Rare case - all values nan
            if np.isnan(mean):
                mean = 0

            dfcpy[col] = dfcpy[col].fillna(mean)
            imputation[col] = mean

        if coltype == "object" or coltype == "category":
            # Replace nan values with mode (most common value) for object/catg columns
            mode = dfcpy[col].mode()

            # Rare case - all values nan (the category one in Hint 4 doesnt make sense
            # since category[0] = nan ...)
            if mode.empty:
                mode = ""
            else:
                mode = mode[0]

            dfcpy[col] = dfcpy[col].fillna(mode)
            imputation[col] = mode

    return dfcpy, imputation

def apply_imputation(df, imputation):
    dfcpy = df.copy()
    for col in df.columns:
        if col == "CLASS" or col == "ID":
            continue

        # Check if this col was imputed
        if col in imputation:
            value = imputation[col] # get imputation (mean or mode)
            dfcpy[col] = dfcpy[col].fillna(value)

    return dfcpy



#NORMALIZATION
def create_normalization(df, normalizationtype='minmax'):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    mapping = {}
    for colname, values in df_copy.iteritems():

        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue
        else :
            # Consider column of type "float" or "int"
            # Normalization of the value
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':
                # dictionary from each column name to a triple ("minmax",min_value,max_value) or ("zscore",mean,std)

                if normalizationtype == "zscore":
                    # z normalization
                    mean = df_copy[colname].mean()
                    std = df_copy[colname].std()
                    #apply z normalization
                    df_copy[colname] = df_copy[colname].apply(lambda x: (x-mean)/std)
                    #filling the dictionary
                    triple = ["zscore", mean, std]
                    mapping[colname] = triple


                else :
                    #min-max normalization
                    min = df_copy[colname].min()
                    max = df_copy[colname].max()
                    # apply min-max normalization
                    df_copy[colname] = [(x-min)/(max-min) for x in df_copy[colname]]
                    # store max,mean and mean for each column
                    # for each column dictionary with three values (nested dictionary)
                    triple = ["minmax", min, max]
                    mapping[colname] = triple

    #print(df_copy)

    return df_copy, mapping

def apply_normalization(df, mapping):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():

        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue;
        else :
            # Consider column of type "float" or "int"
            # Normalization of the value
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':

                if mapping.get(colname)[0] == 'zscore':
                    # z normalization
                    mean = mapping[colname][1]
                    std = mapping[colname][2]
                    #apply z normalization
                    df_copy[colname] = df_copy[colname].apply(lambda x: (x-mean)/std)

                else :
                    #min-max normalization
                    min = mapping[colname][1]
                    max = mapping[colname][2]


                    # apply min-max normalization
                    # limit the output range to [0,1]

                    df_copy[colname] = [1 if (x-min)/(max-min) > 1 else 0 if (x-min)/(max-min) < 0 else (x-min)/(max-min) for x in df_copy[colname]]


    return df_copy


#ONE-HOT
def create_one_hot(df):
    # copy the input dataframe into df_copy and create an empy dictionary
    df_copy = df.copy()
    one_hot = {}
    # only do something if the column is not labeled 'CLASS' or 'ID'
    for colname, values in df_copy.iteritems():
        if colname == "CLASS" or colname == "ID":
            continue
        else :
            # consider column of type "object" or "category"
            if df_copy[colname].dtype == 'category':
                # get the values and put them in the dictionary
                one_hot[colname] = df_copy[colname].unique()
                # create new columns with the one_hot encoding
                for i, val in enumerate(values):
                    # ADDED str() below due to new error (val == float64)
                    df_copy[colname+"_"+str(val)] = df_copy[colname].apply(lambda x: 1 if x==val else 0)
                # drop the old column
                df_copy.drop(colname, axis=1, inplace=True)
    return df_copy, one_hot

def apply_one_hot(df, one_hot):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():
        for key, val in list(one_hot.items()):
            # only do something if the column is already in the dictionary
            if colname == key:
                #create new columns with the one_hot encoding
                for i, v in enumerate(val):
                    df_copy[key+"_"+v] = df_copy[key].apply(lambda x: 1 if x==v else 0)
                #drop the old column
                df_copy.drop(key, axis=1, inplace=True)
    return df_copy



#ACCURACY
def accuracy(df,correctlabels):

    preds = df.idxmax(axis=1) # get index(col) of max value on axis=1(row-wise)
    # print(correctlabels)
    # print(type(correctlabels))

    # print()
    # print(preds)
    # print(type(preds))

    correct = np.sum(preds == correctlabels) # get nr of equal values in two arrays

    acc = correct/len(correctlabels)

    return acc



#BRIER SCORE
def brier_score(df, correctlabels):
    #create matrix of zeros for true classes
    t_class = np.zeros((len(df), len(np.unique(correctlabels))))
    for colname, values in df.iteritems():
        #get the index of the true class
        ind = np.where(df.columns==colname)[0]
        for i, v in enumerate(correctlabels):
            # find the true class and change the values from 0 to 1
            if v == colname:
                t_class[i][ind] = 1
    # brier_score
    brier = np.mean(np.sum((df - t_class)**2, axis=1))
    return brier

# BINNING
def create_bins(df, nobins = 10, bintype="equal-width"):
    # copy the input dataframe into df_copy and create an empty dictionary
    df_copy = df.copy()
    binning = {}
    # only do something if the column is not labeled 'CLASS' or 'ID'
    for colname, values in df_copy.iteritems():
        if colname == "CLASS" or colname == "ID":
            continue;
        else :
            # only consider column of type "float" or "int"
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':
                # create the categories for each value and the tresholds for the bins
                if bintype == "equal-width":
                    cat = pd.cut(df_copy[colname], bins=nobins, labels=False, retbins = True, duplicates = "drop")
                elif bintype == "equal-size":
                    cat = pd.qcut(df_copy[colname], q=nobins, labels=False, retbins = True, duplicates = "drop")
                # replace with the categories and change column type to "category"
                df_copy[colname] = cat[0].astype('category')
                # change the first and the last element of each binning to -np.inf and np.inf
                cat[1][0] = -np.inf
                cat[1][-1] = np.inf
                # add to dictionary
                binning[colname] = cat[1]
    return df_copy, binning



def apply_bins(df, binning):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():
       # only do something if the column is already in the dictionary
       for key, value in list(binning.items()):
            if colname == key:
                # create the category based on the bins provided in binning
                cat = pd.cut(df_copy[colname], bins=binning[colname], labels=False, retbins = False, duplicates = "drop")
                # replace with the categories and change type to "category"
                df_copy[colname] = cat.astype('category')
    return df_copy


#AUC
def auc(df,correctlabels):
    all_dicts = []

    # Create triple dictionaries for each class
    for col in df.columns:

        # Initialize dictionary with s_i --> [0,0]
        triples = {}
        for prob in df[col]:
            triples[prob] = [0,0]

        # Generate dictionary
        i = 0
        for prob in df[col]:
            # number of istances in that class
            if correctlabels[i] == col:
                triples[prob][0] += 1
            # number of istances not in that class
            else:
                triples[prob][1] += 1
            i+=1

        # print(f"Triples for {col}:")
        # print(triples)
        # print()
        all_dicts.append(triples)

    AUCs = [] # list for all AUCs

    # Calculate AUC for each triple dictionary
    for dicts in all_dicts:

        # Calculate total tps and fps
        tot_tp = 0
        tot_fp = 0
        # print("Probability ordering:")
        for prob in sorted(dicts,reverse=True):
            # print(prob)
            pair = dicts[prob]
            tot_tp += pair[0]
            tot_fp += pair[1]
        # print()

        # Begin AUC algorithm
        single_AUC = 0
        cov_tp = 0

        # Loop through the sorted pairs
        for prob in sorted(dicts,reverse=True):
            pair = dicts[prob]
            tp = pair[0]
            fp = pair[1]

            if fp == 0:
                cov_tp += tp
            elif tp == 0:
                single_AUC += (cov_tp/tot_tp) * (fp/tot_fp)
            else:
                single_AUC += (cov_tp/tot_tp) * (fp/tot_fp) + ((tp/tot_tp) * (fp/tot_fp))/2
                cov_tp += tp

        AUCs.append(single_AUC)

    # print("All AUCs:")
    # print(AUCs)
    # print()

    # Count how many times each col appears in correct labels
    col_correct = []
    for col in df.columns:
        correct_count = 0
        for label in correctlabels:
            if col == label:
                correct_count += 1
        # print(f"{col} is correct {correct_count} times")
        col_correct.append(correct_count)

    # Compute final weighted AUC
    AUC = 0
    for i in range(len(AUCs)):
        weight = col_correct[i]/len(correctlabels)
        AUC += weight * AUCs[i]

    # print()
    return AUC


#EUCLIDEAN DISTANCE
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist

## 1. Define the class kNN

In [ ]:
from operator import itemgetter
# Define the class kNN with three functions __init__, fit and predict (after the comments):
class kNN:

# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        self.training_time = None

# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category"
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation,
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
    def fit(self, df, normalizationtype='minmax'):
        dfcpy = df.copy()

        # df_flitered : new df where columns containing only missing values have been dropped
        # col_filter : list of the names of the remaining columns
        df_filtered, col_filter = create_column_filter(dfcpy)
        self.column_filter = col_filter

        # df_imputed : new df missing numeric value in a column has been replaced by the mean of that column
        # missing categoric value in a column has been replaced by the mode of that column imputation
        # impt: mapping (dictionary) from column name to value that has replaced missing values
        df_imputed, impt = create_imputation(df_filtered)
        self.imputation = impt

        # df_normalized: new df where numeric value in a column has been replaced by a normalized value
        # mapping: dictionary from each column name to a triple, consisting of
        # ("minmax",min_value,max_value) or ("zscore",mean,std)
        df_normalized, mapping = create_normalization(df_imputed,'minmax')
        self.normalization = mapping

        #df_onehot: new df where each categoric feature has been replaced by a set of binary features
        # one_hot: dictionary from column name to a set of categories (possible values for the feature)
        # is the hot encoding right because also numerical values are here encoded?
        df_onehot, one_h = create_one_hot(df_normalized)
        self.one_hot = one_h

        #get a series of the column "CLASS" and make it of catgeory type
        categories = dfcpy['CLASS'].astype('category')
        self.training_labels = categories

        # list of class labels
        self.classes = sorted(dfcpy["CLASS"].unique())

        #the values (an ndarray) of the transformed dataframe, i.e., after employing imputation,
        #normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
        data = df_onehot.drop(['CLASS', 'ID'], axis=1)
        self.training_data = data.values


# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
#
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies


    def get_nearest_neighbor_predictions(self, x_test, k):

        #create list for labels for each data point in test set
        op_labels = []
        probs = np.zeros((len(x_test), len(self.classes)))

        #loop through each test data

        for item in x_test:
            #Array to store distances
            point_dist = []

            #Loop through each training Data
            for j in range(len(self.training_data)):
                #Calculating the distance between each item of the test set and each row of the train set
                distances = eucledian(item, np.array(self.training_data[j,:]))
                point_dist.append(distances)
            point_dist = np.array(point_dist)

            #Sorting the array while preserving the index and keeping only the first K datapoints
            dist = np.argsort(point_dist)[:k]

            labels = []
            #get the class of the k datapoints
            for d in dist:
                labels.append(self.training_labels[d])
            op_labels.append(labels)


        for i in range(len(x_test)):
            for d in range(k):
                for c in range(len(self.classes)):
                    if op_labels[i][d] == self.classes[c]:
                        probs[i][c] += 1

        df_probs = pd.DataFrame(probs, columns= self.classes)
        for col, val in df_probs.iteritems():
            df_probs[col] = df_probs[col].apply(lambda x: x/k)

        return df_probs



    def predict(self, df, k):
        dfcpy = df.copy()

        #apply the filters created in the fit function
        df_filtered = apply_column_filter(dfcpy,self.column_filter)
        df_imputed = apply_imputation(df_filtered, self.imputation)
        df_normalized = apply_normalization(df_imputed, self.normalization)
        df_onehot = apply_one_hot(df_normalized, self.one_hot)
        df_onehot = df_onehot.drop(['CLASS','ID'], axis=1)
        self.test_data = df_onehot.values

        predictions = self.get_nearest_neighbor_predictions(self.test_data, k)

        #print("Predictions")
        #print(predictions)
        #print()

        return predictions

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv(filepath + "glass_train.csv")

glass_test_df = pd.read_csv(filepath + "glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

#print(test_labels)

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df, k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions, test_labels), brier_score(predictions, test_labels),
                  auc(predictions, test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.03 s.
Testing time (k=1): 0.18 s.
Testing time (k=3): 0.15 s.
Testing time (k=5): 0.14 s.
Testing time (k=7): 0.16 s.
Testing time (k=9): 0.16 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [ ]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))
print()

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000



## 2. Define the class NaiveBayes

In [ ]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):

# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts

class NaiveBayes :

    def __init__(self):
        self.column_filter = None
        self.binning = None
        self.labels = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None

# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical)
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical)
#                                   value for the feature
#

    # for each column, count
    def class_value_counts(self,df):
        dfcpy = df.copy()

        self.unique_values = {} #dictionary mapping colname with the unique values in the columns
        for colname in dfcpy.columns:
            test = dfcpy[colname].unique()
            self.unique_values[colname] = test

        #dictionnary mapping the column name to a certain combi of value and class
        mapping = {}

        #dictionary mapping colname with the unique values in the columns
        # loop through every column
        for colname in dfcpy.columns:
            if colname == "CLASS" or colname == "ID":
                continue

            # group by column values and class
            mapping_in = {}
            g = dfcpy.groupby([colname,'CLASS'])

            # occurence of the same class and same value of the column
            df_count = g.size()

            #loop through the unique values of each column
            for unique in self.unique_values[colname]:

                # loop through the labels
                for label in self.labels:
                    try:
                        # get_group returns a dataframe if no counts for combination return a key error
                        mapping_in[(unique,label)] = len(g.get_group((unique,label)))
                    except KeyError:
                        mapping_in[(unique,label)] = 0
                mapping[colname] = mapping_in

        return mapping

    # count occurences of class per column
    def class_counts(self, df):
        dfcpy = df.copy()

        mapping = {}
        mapping_in = {}

        # loop through every column
        for colname in dfcpy.columns:
            if colname == "CLASS" or colname == "ID":
                continue

            # group by class
            g = dfcpy.groupby(['CLASS'])

            # loop through the labels
            for label in self.labels:
                #create the inner dictionary
                mapping_in[label] = len(g.get_group((label)))
            #create the outer dictionary
            mapping[colname] = mapping_in

        return mapping


    def fit(self, df, nobins = 10 , bintype = "equal-width"):
        dfcpy = df.copy()

        #filter
        df_filtered, col_filter = create_column_filter(dfcpy)
        self.column_filter = col_filter

        #create binnings
        #self.train_data, binning = create_bins(dfcpy, nobins = 10, bintype = "equal-width")
        self.train_data, binning = create_bins(df_filtered, nobins, bintype)
        self.binning = binning
        #self.train_data = self.train_data.drop(['CLASS','ID'], axis=1)

        #create a list of the class labels of the "CLASS" column
        self.categories = dfcpy['CLASS'].astype('category')
        self.labels = sorted(self.categories.unique())

        # create the mapping from the labels of the "CLASS"
        # to the relative frequencies of the labels
        freq = dfcpy['CLASS'].value_counts() # returns a dataframe of the labels in 'CLASS' associated with their frequency
        mapping_freq = {}
        for i in self.labels:
           mapping_freq[i] = freq[i]/np.sum(freq)
        self.class_priors = mapping_freq

        #create a mapping from the column name to the number of
        #training instances with a specific combination of (value, class)
        # colname -> {(class,value)->nr of occurence ,...}
        # Mg -> {(7,0)->15 ,...}
        self.feature_class_value_counts = self.class_value_counts(self.train_data)

        #create a mapping from the column name to the number of
        #training instances with a specific class label
        # colname -> {class->nr of occurence ,...}
        # Mg -> {7->15 ,...}
        self.feature_class_counts = self.class_counts(self.train_data)

# Input to predict:
# self - the object itself
# df   - a dataframe
#
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and
#         feature_class_counts)
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from
#         a column name (a specific feature) to another mapping, which given a class label and a value for
#         the feature, returns the number of training instances which have included this combination,
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.

    def predict(self, df):
        dfcpy = df.copy()

        # apply col filter and binning
        df_filtered = apply_column_filter(dfcpy, self.column_filter)
        df_binned = apply_bins(df_filtered, self.binning)
        self.test_data = df_binned.drop(['CLASS','ID'], axis=1)

        # hint 2 : likelihood
        likelihood_in={}
        for colname in self.test_data.columns:
            likelihood={}
            for pair in list(self.feature_class_value_counts[colname].keys()):

                class_label = pair[1]

                # nr of (value,class) occurences in col
                pair_occurences = self.feature_class_value_counts[colname][pair]
                # nr of class occurences in col
                class_occurences = self.feature_class_counts[colname][class_label]

                # save relative frequency of observed feature value given the class
                likelihood_in[pair] = pair_occurences/class_occurences

            likelihood[colname] = likelihood_in
            #print(likelihood)

        #hint3
        # Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
        #         product of the relative frequencies
        probs = np.zeros((len(self.test_data), len(self.labels)))
        # for each row
        for id_row, row in self.test_data.iterrows():
            for l, label in enumerate(self.labels):

            # for each class label 1,2,3,5,6,7

                mult = 1
                # for each column RI, ...
                for colname in self.test_data.columns:

                    value = self.test_data.loc[id_row, colname] # get testdata value
                    #print(value)
                    try:
                        mult *= likelihood[colname][(value, label)] # multiply relative freq of value,label
                    except KeyError:
                        mult *= 0

                probs[id_row, l] = mult * self.class_priors[label]

        # hint4
        # prior probability (values) as list
        priors = list(self.class_priors.values())

        # for each row
        for row in range(probs.shape[0]):
            # calculate sum of row
            row_sum = np.sum(probs[row,:])

            # if sum of row nonzero, divide row with sum elemwise
            if row_sum != 0:
                probs[row,:] /= row_sum

            # if sum of row is ZERO, set probs to priors
            else:
                probs[row,:] = priors

        probs_df = pd.DataFrame(probs,columns=self.labels)

        return probs_df

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv(filepath+"glass_train.csv")

glass_test_df = pd.read_csv(filepath+"glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions, test_labels),brier_score(predictions,test_labels),
                   auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                        columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.15 s.
Testing time (3, 'equal-width'): 0.14 s.
Training time (3, 'equal-size'): 0.16 s.
Testing time (3, 'equal-size'): 0.13 s.
Training time (5, 'equal-width'): 0.15 s.
Testing time (5, 'equal-width'): 0.12 s.
Training time (5, 'equal-size'): 0.16 s.
Testing time (5, 'equal-size'): 0.12 s.
Training time (10, 'equal-width'): 0.23 s.
Testing time (10, 'equal-width'): 0.15 s.
Training time (10, 'equal-size'): 0.21 s.
Testing time (10, 'equal-size'): 0.12 s.



'results'

Accuracy  Brier score  AUC
3  equal-width  0.392523     0.719364  0.5
   equal-size   0.392523     0.719364  0.5
5  equal-width  0.392523     0.719364  0.5
   equal-size   0.392523     0.719364  0.5
10 equal-width  0.392523     0.719364  0.5
   equal-size   0.392523     0.719364  0.5

In [ ]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.3178
AUC on training set: 0.5000
Brier score on training set: 0.7658
